In [105]:
words = open("/Users/phucnguyen/PyCharmMiscProject/2.makemore/names.txt").read().splitlines()
chars = sorted(set(''.join(words)))

atoi = {'.': 0}
for index, char in enumerate(chars):
    atoi[char] = index + 1


itoa = {0: '.'}
for index, char in enumerate(chars):
    itoa[index + 1] = char

aatoi = { }
counter = 0
for char in ['.'] + chars:
    for char2 in chars:
        aatoi[char + char2] = counter
        counter += 1

itoaa = { }
counter = 0
for char in ['.'] + chars:
    for char2 in chars:
        itoaa[counter] = char + char2
        counter += 1

E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?


In [107]:
import torch
N = torch.zeros((27,27,27))

# counting version
for word in words:
    normalized = "." + word + "."
    for idx in range(0, len(normalized) - 2):
        a,b,c = atoi[normalized[idx]], atoi[normalized[idx+1]], atoi[normalized[idx+2]]
        N[a][b][c] += 1

# sampling
xs, ys = [], []
for word in words:
    normalized = "." + word + "."
    for idx in range(0, len(normalized) - 2):
        xs.append(aatoi[normalized[idx] + normalized[idx+1]])
        ys.append(atoi[normalized[idx+2]])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

P = (N+1).float()
P /= P.sum(1, keepdim=True)

# Evaluate loss
log_likelyhood = 0.0
for word in words:
    normalized = "." + word + "."
    for idx in range(0, len(normalized) - 2):
        a,b,c = atoi[normalized[idx]], atoi[normalized[idx+1]], atoi[normalized[idx+2]]
        prob = P[a,b,c]
        log_likelyhood += torch.log(prob)

nll = -log_likelyhood
print(f'loss={nll / ys.nelement()}')

loss=2.001167058944702


In [138]:
import torch
import torch.nn.functional as F

# NN version
# sampling
xs, ys = [], []
for word in words:
    normalized = "." + word + "."
    for idx in range(0, len(normalized) - 2):
        xs.append(aatoi[normalized[idx] + normalized[idx+1]])
        ys.append(atoi[normalized[idx+2]])

xs = torch.tensor(xs)
ys = torch.tensor(ys)

W = torch.randn((27 * 26, 27), requires_grad=True)
xenc = F.one_hot(xs, num_classes=27 * 26).float()

In [143]:
learning_rate = 100
epoch = 100

for i in range(epoch):
    # Forward pass
    Wx = xenc @ W
    logits = Wx.exp()
    probs = logits / logits.sum(1, keepdim=True)
    loss = -probs[torch.arange(xs.nelement()), ys].log().mean() + 0.01*(W**2).mean()

    # Backward pass
    W.grad = None
    loss.backward()

    # Update
    W.data += -learning_rate*W.grad

    if i % 10 == 0:
        print(f"iteration={i}, loss={loss.item()}")

iteration=0, loss=2.1461730003356934
iteration=10, loss=2.1445398330688477
iteration=20, loss=2.1429903507232666
iteration=30, loss=2.1415185928344727
iteration=40, loss=2.140118360519409
iteration=50, loss=2.1387853622436523
iteration=60, loss=2.137514352798462
iteration=70, loss=2.1363015174865723
iteration=80, loss=2.1351428031921387
iteration=90, loss=2.1340348720550537


In [266]:
def calc(inp):
    current = inp

    while True:
        last_two = current[-2:]
        position = aatoi[last_two]
        xenc = F.one_hot(torch.tensor([position]), num_classes=27 * 26).float()
        Wx = xenc @ W
        logits = Wx.exp()
        p = logits / logits.sum(1, keepdim=True)
        chr_idx = torch.multinomial(p, 1, replacement=True, generator=g_cpu).item()
        chr = itoa[chr_idx]
        current += chr
        if chr == '.':
            return current

for i in range(5):
    print(calc('ph'))

phivin.
phesian.
phia.
phiry.
phon.


E02: split up the dataset randomly into 80% train set, 10% dev set, 10% test set. Train the bigram and trigram models only on the training set. Evaluate them on dev and test splits. What can you see?



E03: use the dev set to tune the strength of smoothing (or regularization) for the trigram model - i.e. try many possibilities and see which one works best based on the dev set loss. What patterns can you see in the train and dev set loss as you tune this strength? Take the best setting of the smoothing and evaluate on the test set once and at the end. How good of a loss do you achieve?


E04: we saw that our 1-hot vectors merely select a row of W, so producing these vectors explicitly feels wasteful. Can you delete our use of F.one_hot in favor of simply indexing into rows of W?


E05: look up and use F.cross_entropy instead. You should achieve the same result. Can you think of why we'd prefer to use F.cross_entropy instead?


E06: meta-exercise! Think of a fun/interesting exercise and complete it.